In [1]:
import numpy as np
import pandas as pd
import pickle
import sklearn.metrics as met
import lightgbm as lg
import xgboost as xgb

In [2]:
with open('0306_dataset.pickle', 'rb') as handle:
    raw_data,prepare_training = pickle.load(handle)
    
with open('0326_predData.pickle', 'rb') as handle:
    stock_score,stock_pred,max_precision = pickle.load(handle)

##### raw_data : 주식데이터 + 파생변수 \n
##### prepare_training : 이전에 했던 코릴레이션으로 파생변수 선택리스트 \n
##### stock_score : 주식별 Y에 대한 score 정리 (사실상 안씀) \n
##### stock_pred : 주식별 Y별 pred \n
##### max_precision : 주식별 최고성적의 Y에 대한 precision과 profit \n

#  profit을 구하는 알고리즘에 대한 설명 \n
##### 먼저 모델링을 통해 prediction을 하고 train data에 대한 score를 구한다. (lgbm, xgb를 둘다 사용해봐서 코드상엔 lgbm = xgb.model로 되어있음)\n
##### 그다음 test시작일 (2017년 6월 30일)을 기준으로 최근 90일 동안의 precision이 조건에 맞지않으면 -1로 바꿔준다. \n
##### Y_{}_{}에서 첫번째는 가장 precision이 높았던 number이고 두번째는 voting에 사용하기 때문에 전부 사용한다. \n
##### 따라서 아래의 코드의 경우 1가지 날에 대한 pred을 voting할 때 30개 중에서 -1로 분류된것을 제외하고 0,1값만을 사용하여 voting한다 \n
##### 이렇게 최종적으로 pred이 산출된 것을 사용해서 profit을 측정한다.

In [10]:
# 수익률 구하는 함수

def transaction(stock, pred, term1):
    tran_table = pd.DataFrame(columns = ['profit'])
    now_index = 0
    tran_table.loc['2017-06-30'] = 0
    while now_index <= len(pred):
        if pred.iloc[now_index] > 0:
            tran_table.loc[pred.index[now_index+term1]] = raw_data[str(stock)]['Y1_{:02}'.format(term1)].iloc[now_index]
            now_index = now_index + term1
        else:
            now_index = now_index + 1

        if now_index + term1 >= len(pred):
            break

    tran_table = tran_table+1
    tran_table = tran_table.cumprod()
    
    return tran_table

# 1 .모델링 코드

In [ ]:
stock_pred_pram = {}

main_epoc = pd.DataFrame()
for stock in raw_data.keys():
    i = int(max_precision['number'].loc[stock])
    main_epoc = pd.DataFrame()
    for j in np.arange(1,90,3):
        column_list = []
        for col in prepare_training[stock]['Y2_{:02}_{:02}'.format(i,j)].keys():
            column_list.append(col)
        lgbm = xgb.XGBRegressor(random_state=0,n_jobs=-1)
        X_train = raw_data[stock][column_list][:'2017-06-30']
        y_train = raw_data[stock]['Y2_{:02}_{:02}'.format(i,j)][:'2017-06-30']
        X_test = raw_data[stock][column_list]['2017-06-30':'2018-06-30']
        y_test = raw_data[stock]['Y1_{:02}'.format(i)]['2017-06-30':'2018-06-30']
        lgbm.fit(X_train, y_train)
        over_term_pred = pd.concat([X_train['2017-01-01':],X_test['2017-06-30':'2018-06-30']],axis=0)
        a = lgbm.predict(over_term_pred)
        a =  np.where(a > 0,1,0)
        epoc_series = pd.DataFrame(a, columns = ['{}_pred'.format(j)], index = over_term_pred.index)
        main_epoc = pd.concat([main_epoc,epoc_series],axis=1)
    stock_pred_pram[stock] = main_epoc

# 2. 최근 precision을 보고 pred을 믿을지 말지 판단 일정 수준 이하면 해당 pred을 voting에 사용하지 않는다.

In [ ]:
for stock in raw_data.keys():
    i = int(max_precision['number'][stock])
    stock_pred_copy = stock_pred_pram[stock].copy()
    for j in np.arange(1,90,3):

        raw_Y = raw_data[stock]['Y1_{:02}'.format(int(max_precision['number'][stock]))]
        raw_Y = raw_Y[stock_pred_pram[stock].index]
        real_Y = pd.DataFrame(np.where(raw_Y > 0,1,0), index = raw_Y.index, columns = [raw_Y.name])

        column_list = []
        for col in prepare_training[stock]['Y2_{:02}_{:02}'.format(i,j)].keys():
            column_list.append(col)

        xgboost_model = xgb.XGBRegressor(random_state=0,n_jobs=-1)
        X_train = raw_data[stock][column_list][:'2017-06-30']
        y_train = raw_data[stock]['Y2_{:02}_{:02}'.format(i,j)][:'2017-06-30']
        xgboost_model.fit(X_train, y_train)
        train_pred = xgboost_model.predict(raw_data[stock][column_list][:'2017-06-30'])
        main_score = met.precision_score(np.where(train_pred > 0,1,0), np.where(raw_data[stock]['Y1_{:02}'.format(i)][:'2017-06-30'] > 0,1,0))

        k = 0
        while 90 + k <= len(raw_Y):
            if met.precision_score(stock_pred_copy['{}_pred'.format(j)].iloc[k:90+k],real_Y.iloc[k:90+k]) < 0.5:
                stock_pred_pram[stock]['{}_pred'.format(j)].iloc[89+k] = -1
                
            elif met.precision_score(stock_pred_copy['{}_pred'.format(j)].iloc[k:90+k],real_Y.iloc[k:90+k]) < main_score*0.9:
                stock_pred_pram[stock]['{}_pred'.format(j)].iloc[89+k] = -1
            k = k + 1

# 3. 위에서 최종적으로 pred을 0,1,-1로 분류하여 -1인 것을 제외하고 voting을 통해 최종 pred을 결정한다.

In [ ]:
votng_df = pd.DataFrame(index = stock_pred_pram[stock]['2017-06-30':].iloc[1:].index,columns = list(raw_data.keys()))

for stock in raw_data.keys():
    datas = stock_pred_pram[stock]['2017-06-30':].iloc[1:]
    for ind in datas.index:
        a = datas.loc[ind].values
        if len(a[a >=0]) == 0:
            votng_df[stock].loc[ind] = 0
        
        elif a[a >= 0].sum()/len(a[a >= 0]) >= 0.5:
            votng_df[stock].loc[ind] = 1
            
        else:
            votng_df[stock].loc[ind] = 0
            
profit_dict = {}

for stock in raw_data.keys():
    pred = votng_df[stock]
    i = int(max_precision['number'][stock])
    profit_dict[stock] = transaction(stock, pred, i)

    indexing = []
    for i in range(len(profit_dict[stock])):
        indexing.append(str(profit_dict[stock].index[i])[:10])
    profit_dict[stock].index = indexing
    
profit_df = pd.DataFrame(columns = ['profit'])

for stock in raw_data.keys():
    profit_df.loc[stock] = float(profit_dict[stock].iloc[len(profit_dict[stock])-1])